# Data analysis with TogoDX: Network visualization

TogoDX/Human (https://togodx.dbcls.jp/human) の生物学データ解析への活用例。文責: 大田 (DBCLS)}

## Background

TogoDX は、 RDF 化され知識グラフとして統合された生命医科学データにアクセスするためのインターフェースである。ユーザはデータセット全体、あるいはユーザがIDのリストとして与えたサブセットに対して、統合された属性を用いてフィルタリング (filter) や属性情報の追加 (map attributes) を行い、次のアクションの対象となるデータセットを抽出することができる。

ただし、TogoDX によって出力されたデータは primary dataset (ユーザが選択する統合の軸となるデータセット) をハブとして複数の属性が結合された星型グラフの集合である。出力テーブルにおける行間（複数の星型グラフ）やカラム間 (星型グラフにおける葉ノード間）の結合は保証されていない。抽出したデータはロング型の TSV として出力・ダウンロードが可能である。

## User story

- ユーザはベンチ・サイエンティストである。何かしらの分子を定量する実験を行い、群間比較を行う。結果として興味のある分子のリストを得る。
- リストに含まれる分子はそれぞれ何かしらの定量値を持っているが、分子間の関係はわからない。
- TogoDX を用いて得られた分子のリストに知識グラフの情報を付加する。
- それぞれが独立したリスト形式のデータに対してネットワーク構造を持つ知識グラフの情報を与えることで、グラフとして扱うことができる。
- グラフの状態で可視化やグラフ分析を行うことで、ハブとなる分子、クラスターを構成する分子群などを捉えることで、分子間の関係性を説明する。

## Procedure

### TogoDX での操作

- TogoDX/Human に分子IDのリストを入力し map your IDs を実行する。
    - 入力されたIDに対応するエントリと、そこに接続するデータセットの属性にピンが刺さる。
- (optional) リストのIDの数が多い場合には Add filters 機能を実行する。
    - たとえば gene list に対して chromosome や gene type で絞り込む。
- Map attributes 機能で、与えたIDに対して興味のある属性情報を追加する。
    - たとえば gene list に対して protein structure, pathway, drug などの属性を追加する。
- View results で得られたデータを確認する。問題がなければ TSV をダウンロードする。
    - あまりに数が多いと後の解析が大変なので戻ってフィルタで減らすか覚悟を決める。

### Notebook での操作

- 分析に必要なパッケージ群をインストールし、ロードする。
- TogoDX からダウンロードしたTSVを読み込む。
    - 統計値などを取得し読み込んだテーブルの規模や属性の分布を把握する。
- (optional) 前処理を行う。
    - ダウンロードした TSV の1行はハブとなる primary dataset の1エントリと、それに対応する属性を持つ1エントリの 1-1 ペアである。
    - 下の例では、その後操作しやすいように行を識別する pair ID を生成するなどの簡単な前処理を行っている。
- TogoID を用いて星型グラフにおける葉ノード間を接続する。
    - TogoID API を利用して、葉ノード同士を接続するリンクを取得、グラフに追加する。
- 得られたグラフを可視化する。

## Further analysis

可視化されたグラフに対しては解析者のアイディア次第で様々な解析が可能である。例として以下のようなアクションが想定される。

- サブグラフの規模によってフィルタし、独立したグラフに存在する分子を特定する。
- 追加された属性に対して任意の weight を与えることで、より重要なパスを特定し、そこに関連する分子を特定する。
- グラフの中心性を求める手法を用いてハブとなる分子を特定する。

### Limitation

上で想定する解析における既知の問題は以下。

- ダウンロードされた TSV において target dataset のラベル情報が欠けている。
    - TSV生成の実装における問題なので技術的に解消が可能。
- TogoID API のスケーラビリティ上限が未知
    - 非常に短い間隔で叩き続けるとレスポンスが返ってこない可能性がある。
    - 下に示した実装の例は非効率なので、改善できる可能性がある。


## Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
from scipy import stats

In [2]:
#pd.set_option('display.max_rows', 100)

In [3]:
import sys
from urllib.request import urlopen
import json
import time

In [4]:
!{sys.executable} -m pip install pyyaml --quiet
import yaml

In [5]:
!{sys.executable} -m pip install pyvis --quiet
# https://pyvis.readthedocs.io/
from pyvis.network import Network

## Load dataset

In [6]:
data_path = "../data/togodx-20230217-12655.tsv"
d = pd.read_table(data_path)
d

,orig_dataset,orig_entry,orig_label,dest_dataset,dest_entry,node,value
0,ensembl_gene,ENSG00000172915,NBEA,ensembl_transcript,ENST00000379939,transcript_biotype_ensembl,protein coding
1,ensembl_gene,ENSG00000172915,NBEA,ensembl_transcript,ENST00000400445,transcript_biotype_ensembl,protein coding
2,ensembl_gene,ENSG00000172915,NBEA,ensembl_transcript,ENST00000537702,transcript_biotype_ensembl,protein coding
3,ensembl_gene,ENSG00000172915,NBEA,ensembl_transcript,ENST00000629018,transcript_biotype_ensembl,protein coding
4,ensembl_gene,ENSG00000172915,NBEA,ncbigene,26960,gene_evolutionary_conservation_homologene,"Insect, Worm"
...,...,...,...,...,...,...,...
4529,ensembl_gene,ENSG00000157933,SKI,uniprot,P12755,protein_disease_related_proteins_uniprot,Disease variant
4530,ensembl_gene,ENSG00000157933,SKI,uniprot,P12755,structure_data_existence_uniprot,Proteins with structure data
4531,ensembl_gene,ENSG00000157933,SKI,uniprot,P12755,interaction_proteins_in_pathway_reactome,Signaling Pathways
4532,ensembl_gene,ENSG00000157933,SKI,uniprot,P12755,interaction_proteins_in_pathway_reactome,Gene expression (Transcription)


In [7]:
d.describe()

,orig_dataset,orig_entry,orig_label,dest_dataset,dest_entry,node,value
count,4534,4534,4534,4534,4534,4534,4534
unique,1,154,154,4,1894,8,99
top,ensembl_gene,ENSG00000044115,CTNNA1,uniprot,836,interaction_proteins_in_pathway_reactome,protein coding
freq,4534,126,126,2528,19,903,885


In [8]:
# List up the selected attributes
# d['node'].unique()

## Extract subset

手法の確認のためにまず１行目だけを取り出して可視化する。つまり、星型グラフ１個について情報を付加する。

In [9]:
first_row_orig_id = d.head(1)['orig_entry'][0]
first_row_orig_id

'ENSG00000172915'

In [10]:
orig_id = first_row_orig_id
subset = d[d['orig_entry'] == orig_id]
# Add columns of identifiers in dataset:id form
subset['orig_id'] = subset['orig_dataset'] + ':' + subset['orig_entry']
subset['dest_id'] = subset['dest_dataset'] + ':' + subset['dest_entry']
subset

/tmp/ipykernel_82/1901048836.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['orig_id'] = subset['orig_dataset'] + ':' + subset['orig_entry']
/tmp/ipykernel_82/1901048836.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['dest_id'] = subset['dest_dataset'] + ':' + subset['dest_entry']


,orig_dataset,orig_entry,orig_label,dest_dataset,dest_entry,node,value,orig_id,dest_id
0,ensembl_gene,ENSG00000172915,NBEA,ensembl_transcript,ENST00000379939,transcript_biotype_ensembl,protein coding,ensembl_gene:ENSG00000172915,ensembl_transcript:ENST00000379939
1,ensembl_gene,ENSG00000172915,NBEA,ensembl_transcript,ENST00000400445,transcript_biotype_ensembl,protein coding,ensembl_gene:ENSG00000172915,ensembl_transcript:ENST00000400445
2,ensembl_gene,ENSG00000172915,NBEA,ensembl_transcript,ENST00000537702,transcript_biotype_ensembl,protein coding,ensembl_gene:ENSG00000172915,ensembl_transcript:ENST00000537702
3,ensembl_gene,ENSG00000172915,NBEA,ensembl_transcript,ENST00000629018,transcript_biotype_ensembl,protein coding,ensembl_gene:ENSG00000172915,ensembl_transcript:ENST00000629018
4,ensembl_gene,ENSG00000172915,NBEA,ncbigene,26960,gene_evolutionary_conservation_homologene,"Insect, Worm",ensembl_gene:ENSG00000172915,ncbigene:26960
...,...,...,...,...,...,...,...,...,...
75,ensembl_gene,ENSG00000172915,NBEA,uniprot,A0A8I5QKR1,interaction_proteins_in_pathway_reactome,Not in the pathway,ensembl_gene:ENSG00000172915,uniprot:A0A8I5QKR1
76,ensembl_gene,ENSG00000172915,NBEA,uniprot,A0A8I5QKR6,interaction_proteins_in_pathway_reactome,Not in the pathway,ensembl_gene:ENSG00000172915,uniprot:A0A8I5QKR6
77,ensembl_gene,ENSG00000172915,NBEA,uniprot,Q5T321,interaction_proteins_in_pathway_reactome,Not in the pathway,ensembl_gene:ENSG00000172915,uniprot:Q5T321
78,ensembl_gene,ENSG00000172915,NBEA,uniprot,Q8NFP9,interaction_proteins_in_pathway_reactome,Not in the pathway,ensembl_gene:ENSG00000172915,uniprot:Q8NFP9


In [11]:
# Generate ID pairs that connect each other
id_pairs = subset[['orig_id', 'dest_id']].drop_duplicates()
id_pairs = id_pairs[id_pairs['orig_id'] != id_pairs['dest_id']]
id_pairs

,orig_id,dest_id
0,ensembl_gene:ENSG00000172915,ensembl_transcript:ENST00000379939
1,ensembl_gene:ENSG00000172915,ensembl_transcript:ENST00000400445
2,ensembl_gene:ENSG00000172915,ensembl_transcript:ENST00000537702
3,ensembl_gene:ENSG00000172915,ensembl_transcript:ENST00000629018
4,ensembl_gene:ENSG00000172915,ncbigene:26960
8,ensembl_gene:ENSG00000172915,uniprot:A0A0D9SF28
9,ensembl_gene:ENSG00000172915,uniprot:A0A8I5KQL6
10,ensembl_gene:ENSG00000172915,uniprot:A0A8I5KQP5
11,ensembl_gene:ENSG00000172915,uniprot:A0A8I5KRX1
12,ensembl_gene:ENSG00000172915,uniprot:A0A8I5KRZ1


## Visualization

### Configuration

In [12]:
attributes_json_url = 'https://github.com/togodx/togodx-config-human/raw/develop/config/attributes.dx-server.json'
attributes_json = json.loads(urlopen(attributes_json_url).read())

In [13]:
togoid_dataset_config_url = 'https://github.com/togoid/togoid-config/raw/main/config/dataset.yaml'
dataset_config = yaml.safe_load(urlopen(togoid_dataset_config_url).read())

In [14]:
# Color code
categories_color = {
  "Analysis": { "color": "#696969" },
  "Compound": { "color": "#a853c6" },
  "Disease": { "color": "#5361c6" },
  "Domain": { "color": "#a2c653" },
  "Experiment": { "color": "#696969" },
  "Function": { "color": "#696969" },
  "Gene": { "color": "#53c666" },
  "Glycan": { "color": "#673aa6" },
  "Interaction": { "color": "#c65381" },
  "Literature": { "color": "#696969" },
  "Ortholog": { "color": "#53c666" },
  "Pathway": { "color": "#c65381" },
  "Probe": { "color": "#53c666" },
  "Project": { "color": "#696969" },
  "Protein": { "color": "#a2c653" },
  "Reaction": { "color": "#c65381" },
  "Sample": { "color": "#696969" },
  "SequenceRun": { "color": "#696969" },
  "Structure": { "color": "#c68753" },
  "Submission": { "color": "#696969" },
  "Taxonomy": { "color": "#006400" },
  "Transcript": { "color": "#53c666" },
  "Variant": { "color": "#53c3c6" },
};

dataset_color = {}
for ds in dataset_config:
    cat = dataset_config[ds]['category']
    if not cat in categories_color:
        continue
    dataset_color[ds] = categories_color[cat]['color']

dataset_color

{'affy_probeset': '#53c666',
 'bioproject': '#696969',
 'biosample': '#696969',
 'ccds': '#53c666',
 'chebi': '#a853c6',
 'chembl_compound': '#a853c6',
 'chembl_target': '#a2c653',
 'civic_gene': '#53c666',
 'clinvar': '#53c3c6',
 'dbsnp': '#53c3c6',
 'dgidb': '#c65381',
 'doid': '#5361c6',
 'drugbank': '#a853c6',
 'ec': '#696969',
 'ena': '#53c666',
 'ensembl_gene': '#53c666',
 'ensembl_protein': '#a2c653',
 'ensembl_transcript': '#53c666',
 'flybase_gene': '#53c666',
 'gea': '#696969',
 'glytoucan': '#673aa6',
 'go': '#696969',
 'hgnc': '#53c666',
 'hgnc_symbol': '#53c666',
 'hmdb': '#a853c6',
 'homologene': '#53c666',
 'cog': '#53c666',
 'hp': '#5361c6',
 'human_protein_atlas': '#a2c653',
 'inchi_key': '#a853c6',
 'insdc': '#53c666',
 'insdc_master': '#696969',
 'intact': '#c65381',
 'interpro': '#a2c653',
 'kegg_compound': '#a853c6',
 'kegg_disease': '#5361c6',
 'kegg_orthology': '#53c666',
 'kegg_pathway': '#c65381',
 'kegg_reaction': '#c65381',
 'lrg': '#53c666',
 'mbgd_gene': '#

### PyVis init

In [15]:
net = Network(notebook=True, cdn_resources='in_line', bgcolor='gray', font_color='white')

### Add hub node (primary dataset) and connected nodes (attributes added by togodx)

In [16]:
for oid in id_pairs['orig_id'].drop_duplicates():
    net.add_node(oid, color='white')

In [17]:
for did in id_pairs['dest_id'].drop_duplicates():
    ds = did.split(':')[0]
    color = dataset_color[ds]
    net.add_node(did, color=color)

In [18]:
net.add_edges(id_pairs.to_numpy())

In [19]:
net.toggle_physics(True)
net.show('mygraph.html')

In [20]:
outer_entries = subset[subset['dest_dataset'] != subset['orig_dataset'][0] ][['dest_dataset', 'dest_entry']].drop_duplicates()
# outer_entries

In [21]:
datasets_dict = attributes_json['datasets']
# datasets_dict

In [22]:
# Connecting outer nodes (!= primary dataset, the network hub node)
for index, row in outer_entries.iterrows():
    for target in outer_entries['dest_dataset'].unique():
        # Loop for the datasets other than itself
        if target != row['dest_dataset']:
            time.sleep(1)
            print('from: ' + row['dest_dataset'] + ' to: ' + target)
            
            config_dataset = datasets_dict[row['dest_dataset']]['conversion']
            if not target in config_dataset:
                continue

            togoid_api_route = config_dataset[target]
            api_url = togoid_api_route + row['dest_entry']
            print(api_url)
            
            res_json = urlopen(api_url)
            res = json.loads(res_json.read())

            n_from = row['dest_dataset'] + ':' + row['dest_entry']
            for i in res['results']:
                n_to = target + ':' + i
                print('Edge from:' + n_from + ', to: ' + n_to)
                net.add_node(n_to)
                net.add_edge(n_from, n_to)

from: ensembl_transcript to: ncbigene
https://api.togoid.dbcls.jp/convert?format=json&route=ensembl_transcript,ncbigene&ids=ENST00000379939
Edge from:ensembl_transcript:ENST00000379939, to: ncbigene:26960
from: ensembl_transcript to: uniprot
https://api.togoid.dbcls.jp/convert?format=json&route=ensembl_transcript,uniprot&ids=ENST00000379939
Edge from:ensembl_transcript:ENST00000379939, to: uniprot:Q5T321
from: ensembl_transcript to: mondo
https://api.togoid.dbcls.jp/convert?format=json&route=ensembl_transcript,ncbigene,medgen,mondo&ids=ENST00000379939
Edge from:ensembl_transcript:ENST00000379939, to: mondo:0030930
from: ensembl_transcript to: ncbigene
https://api.togoid.dbcls.jp/convert?format=json&route=ensembl_transcript,ncbigene&ids=ENST00000400445
Edge from:ensembl_transcript:ENST00000400445, to: ncbigene:26960
from: ensembl_transcript to: uniprot
https://api.togoid.dbcls.jp/convert?format=json&route=ensembl_transcript,uniprot&ids=ENST00000400445
Edge from:ensembl_transcript:ENST00

from: uniprot to: ensembl_transcript
from: uniprot to: ncbigene
https://api.togoid.dbcls.jp/convert?format=json&route=uniprot,ncbigene&ids=A0A8I5QKR6
from: uniprot to: mondo
https://api.togoid.dbcls.jp/convert?format=json&route=uniprot,ncbigene,medgen,mondo&ids=A0A8I5QKR6
from: uniprot to: ensembl_transcript
from: uniprot to: ncbigene
https://api.togoid.dbcls.jp/convert?format=json&route=uniprot,ncbigene&ids=Q5T321
Edge from:uniprot:Q5T321, to: ncbigene:26960
from: uniprot to: mondo
https://api.togoid.dbcls.jp/convert?format=json&route=uniprot,ncbigene,medgen,mondo&ids=Q5T321
Edge from:uniprot:Q5T321, to: mondo:0030930
from: uniprot to: ensembl_transcript
from: uniprot to: ncbigene
https://api.togoid.dbcls.jp/convert?format=json&route=uniprot,ncbigene&ids=Q8NFP9
Edge from:uniprot:Q8NFP9, to: ncbigene:26960
from: uniprot to: mondo
https://api.togoid.dbcls.jp/convert?format=json&route=uniprot,ncbigene,medgen,mondo&ids=Q8NFP9
Edge from:uniprot:Q8NFP9, to: mondo:0030930
from: mondo to: ens

In [23]:
net.toggle_physics(True)
net.show('mygraph.html')